In [ ]:

!git clone https://github.com/zsef123/Connected_components_PyTorch.git
%cd Connected_components_PyTorch
!pip install -e .

!pip install -q opencv-python matplotlib



Cloning into 'Connected_components_PyTorch'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 7), reused 23 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 132.96 KiB | 7.82 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/Connected_components_PyTorch
Obtaining file:///content/Connected_components_PyTorch
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cc_torch


In [2]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from matplotlib import pyplot as plt
from PIL import Image
from cc_torch import connected_components
from google.colab import files



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



img_path = "/content/drive/MyDrive/img/p1/RC04844/page_1.png"


Using device: cuda


In [ ]:
gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


upscaled = cv2.resize(gray, None, fx=8, fy=8, interpolation=cv2.INTER_LANCZOS4)
clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(4,4))
enhanced = clahe.apply(upscaled)


def phansalkar_threshold(image, window_size=31, k=0.2, R=128, p=20, q=6.2):
    image = image.astype(np.float32)
    mean = cv2.boxFilter(image, ddepth=-1, ksize=(window_size, window_size))
    sqmean = cv2.sqrBoxFilter(image, ddepth=-1, ksize=(window_size, window_size))
    stddev = np.sqrt(np.maximum(sqmean - mean**2, 1e-4))
    threshold = mean * (1 + p * np.exp(-q * mean / 255) + k * ((stddev / R) - 1))
    binary = (image > threshold).astype(np.uint8)
    return binary

binary = phansalkar_threshold(enhanced)

def remove_speckle(binary_np, kernel_size=5, iterations=2):
    inv = cv2.bitwise_not(binary_np * 255)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    opened = cv2.morphologyEx(inv, cv2.MORPH_OPEN, kernel, iterations=iterations)
    return (cv2.bitwise_not(opened) > 128).astype(np.uint8)

binary_clean = remove_speckle(binary)


In [ ]:
# Visualize intermediate binary_clean
#cv2.imwrite("binary_clean.png", binary_clean * 255)
#files.download("binary_clean.png")

In [5]:
binary = binary_clean * 255
final = cv2.resize(binary, (gray.shape[1], gray.shape[0]), interpolation=cv2.INTER_AREA)
cv2.imwrite("final.png", final)
files.download("final.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ✅ Step 6: GPU-accelerated connected components
tensor = torch.tensor(binary_clean).to(device, dtype=torch.uint8) # Changed shape here
print("Shape of tensor:", tensor.shape)
labels = connected_components_labeling(tensor) # shape: [H, W]






In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from cc_torch import connected_components_labeling

def _pad_to_even_2d(t: torch.Tensor, pad_value: int = 0, mode: str = "constant"):
    assert t.ndim in (2, 3)
    if t.ndim == 2:
        H, W = t.shape
        pad = (0, W % 2, 0, H % 2)
        if pad == (0,0,0,0):
            return t, (slice(0,H), slice(0,W))
        tp = F.pad(t[None, None], pad, mode=mode, value=pad_value).squeeze(0).squeeze(0)
        return tp, (slice(0,H), slice(0,W))
    else:
        H, W, C = t.shape
        pad = (0, W % 2, 0, H % 2)
        if pad == (0,0,0,0):
            return t, (slice(0,H), slice(0,W), slice(0,C))
        tp = F.pad(t.permute(2,0,1)[None], pad, mode=mode, value=pad_value).squeeze(0).permute(1,2,0)
        return tp, (slice(0,H), slice(0,W), slice(0,C))

def _dilate(mask: torch.Tensor, iters: int) -> torch.Tensor:
    """Binary dilation using max-pool; mask is [H,W] bool/uint8 on GPU."""
    if iters <= 0:
        return mask
    x = mask.float()[None, None]  # [1,1,H,W]
    for _ in range(iters):
        x = (F.max_pool2d(x, kernel_size=3, stride=1, padding=1) > 0).float()
    return (x[0,0] > 0)

def apply_regions_to_original_no_blend_with_halo(
    original_img_np: np.ndarray,   # uint8, [H,W] or [H,W,3]
    binary_np_img: np.ndarray,     # uint8 {0,1}, [H,W]
    min_area_black: int = 30,      # remove tiny black speckles
    min_area_white: int = 30,      # fill tiny white holes
    fg_val: int = 0,               # black for grayscale
    bg_val: int = 255,             # white for grayscale
    speckle_expand_iters: int = 1, # grow radius in pixels around speckle
    speckle_expand_gray_thresh: int | None = 210  # e.g., 200–220; None = no gating
) -> np.ndarray:
    """
    Detect on binary; apply on original.
    Black speckles: set speckle + (optional light-gray halo) to white.
    White holes: set small 1-components to black.
    Everything else unchanged.
    """
    assert binary_np_img.ndim == 2
    assert set(np.unique(binary_np_img)).issubset({0,1})
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type != 'cuda':
        raise RuntimeError("cc_torch build requires CUDA. Enable GPU or use a CPU CCL fallback.")

    # tensors
    bin_t = torch.from_numpy(binary_np_img.astype(np.uint8)).to(device)
    if original_img_np.ndim == 2:
        orig_t = torch.from_numpy(original_img_np).to(device)           # [H,W]
        is_color = False
    else:
        orig_t = torch.from_numpy(original_img_np).to(device)           # [H,W,3]
        is_color = True

    # pad once
    bin_pad, _ = _pad_to_even_2d(bin_t, pad_value=0, mode="constant")
    orig_pad, unpad_img = _pad_to_even_2d(orig_t, pad_value=0, mode="replicate")
    H, W = bin_pad.shape[:2]
    assert (H, W) == orig_pad.shape[:2]

    # -------- Pass 1: remove black speckles (+ optional halo)
    inv = (1 - bin_pad)  # speckles -> 1
    labels_black = connected_components_labeling(inv).to(device)

    for lid in torch.unique(labels_black):
        if lid.item() == 0:
            continue
        region = (labels_black == lid)
        if int(region.sum().item()) >= min_area_black:
            continue

        # expand region by N iters
        expanded = _dilate(region, speckle_expand_iters)

        # optionally gate expansion by original gray level (light-gray only)
        if speckle_expand_gray_thresh is not None:
            if is_color:
                # use luminance approximation to gate
                # convert BGR/RGB ambiguity not critical for thresholding halo.
                y = (0.299*orig_pad[...,0] + 0.587*orig_pad[...,1] + 0.114*orig_pad[...,2]).to(orig_pad.dtype)
                gate = (1 <= speckle_expand_gray_thresh)
            else:
                gate = (1 <= speckle_expand_gray_thresh)
            expanded = expanded & gate

        # always include the original speckle core
        expanded = expanded | region

        # set expanded area to background white
        if is_color:
            orig_pad[..., 0][expanded] = bg_val
            orig_pad[..., 1][expanded] = bg_val
            orig_pad[..., 2][expanded] = bg_val
        else:
            orig_pad[expanded] = bg_val

    # -------- Pass 2: fill white holes (no expansion)
    labels_white = connected_components_labeling(bin_pad).to(device)
    for lid in torch.unique(labels_white):
        if lid.item() == 0:
            continue
        region = (labels_white == lid)
        if int(region.sum().item()) < min_area_white:
            if is_color:
                orig_pad[..., 0][region] = fg_val
                orig_pad[..., 1][region] = fg_val
                orig_pad[..., 2][region] = fg_val
            else:
                orig_pad[region] = fg_val

    # crop back
    cleaned = orig_pad[unpad_img].detach().cpu().numpy()
    return cleaned





In [ ]:
img_path = "/content/drive/MyDrive/img/p1_upload/RC04848/page_2.png"
gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
binary = (gray > 127).astype(np.uint8)

cleaned_gray = apply_regions_to_original_no_blend_with_halo(
    original_img_np=gray,
    binary_np_img=binary,
    min_area_black=3,
    min_area_white=2,
    speckle_expand_iters=1,          # try 1–2
    speckle_expand_gray_thresh=0   # None for pure geometric; lower -> stricter
)
# Show
plt.figure(figsize=(15,4))
plt.subplot(1,3,1); plt.title("Original Gray");  plt.imshow(gray, cmap='gray'); plt.axis('off')
plt.subplot(1,3,2); plt.title("Binary (detect)");plt.imshow(binary*255, cmap='gray'); plt.axis('off')
plt.subplot(1,3,3); plt.title("Cleaned Gray");   plt.imshow(cleaned_gray, cmap='gray'); plt.axis('off')
plt.tight_layout(); plt.show()

In [ ]:
cv2.imwrite("cleaned.png", cleaned_gray)
files.download("cleaned.png")

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive



# Define input and output directories
input_dir = '/content/drive/MyDrive/img'
output_dir = '/content/drive/MyDrive/output_img'

# Phansalkar thresholding
def phansalkar_threshold(image, window_size=31, k=0.2, R=128, p=2.0, q=6.2):
    image = image.astype(np.float32)
    mean = cv2.boxFilter(image, ddepth=-1, ksize=(window_size, window_size))
    sqmean = cv2.sqrBoxFilter(image, ddepth=-1, ksize=(window_size, window_size))
    stddev = np.sqrt(np.maximum(sqmean - mean**2, 1e-4))
    threshold = mean * (1 + p * np.exp(-q * mean / 255) + k * ((stddev / R) - 1))
    binary = (image > threshold).astype(np.uint8)
    return binary

# Speckle removal
def remove_speckle(binary_np, kernel_size=5, iterations=2):
    inv = cv2.bitwise_not(binary_np * 255)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    opened = cv2.morphologyEx(inv, cv2.MORPH_OPEN, kernel, iterations=iterations)
    return (cv2.bitwise_not(opened) > 128).astype(np.uint8)

# Image processing pipeline
def process_image(img_path):
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    upscaled = cv2.resize(gray, None, fx=8, fy=8, interpolation=cv2.INTER_LANCZOS4)
    clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(4, 4))
    enhanced = clahe.apply(upscaled)
    binary = phansalkar_threshold(enhanced)
    binary_clean = remove_speckle(binary)
    binary = binary_clean * 255
    final = cv2.resize(binary, (gray.shape[1], gray.shape[0]), interpolation=cv2.INTER_AREA)
    return final

# Create output folder structure
def create_output_structure(input_dir, output_dir):
    for root, dirs, _ in os.walk(input_dir):
        for dir_name in dirs:
            rel_path = os.path.relpath(os.path.join(root, dir_name), input_dir)
            os.makedirs(os.path.join(output_dir, rel_path), exist_ok=True)

# Process and save all images
def process_and_save_images(input_dir, output_dir):
    create_output_structure(input_dir, output_dir)
    for root, _, files in os.walk(input_dir):
        for file_name in files:
            if file_name.lower().endswith('.png'):
                img_path = os.path.join(root, file_name)
                try:
                    processed_img = process_image(img_path)
                    rel_path = os.path.relpath(img_path, input_dir)
                    output_path = os.path.join(output_dir, rel_path)
                    cv2.imwrite(output_path, processed_img)
                    print(f"Saved: {output_path}")
                except Exception as e:
                    print(f"Failed: {img_path} — {e}")

#  Run the batch processor
process_and_save_images(input_dir, output_dir)
